In [6]:
import habanero
import csv
from pybtex.database.input import bibtex

In [7]:
def dois_from_csv(file_path):
    # Read DOIs from a CSV file
    dois = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            dois.extend(row)
    return dois

In [3]:
def fetch_metadata(doi):
    # Fetch metadata for a single DOI using CrossRef
    cr = habanero.Crossref()
    try:
        response = cr.works(ids=doi)
        return response['message']
    except habanero.exceptions.RequestError as e:
        print(f"Error fetching metadata for DOI {doi}: {e}")
        return None

In [4]:
def generate_ris(entries):
    # Generate RIS formatted entries
    bib_data = bibtex.Parser().parse_string(entries)
    writer = bibtex.Writer()
    return writer.to_string(bib_data)

In [5]:
def dois_to_ris(doils):
    # Fetch metadata for each DOI and create a RIS file
    entries = []
    for doi in doils:
        metadata = fetch_metadata(doi)
        if metadata:
            entry = habanero.cn.content_negotiation(ids=doi, format="bibentry")
            entries.append(entry)
    
    ris_entries = '\n'.join(entries)
    ris_data = generate_ris(ris_entries)
    return ris_data

In [8]:
dois = dois_from_csv("doi_set.csv")

In [9]:
# Generate RIS file from DOIs
ris_content = dois_to_ris(dois)

# Write RIS content to a file
with open("metadata_entries.ris", "w", encoding="utf-8") as file:
    file.write(ris_content)

HTTPError: 404 Client Error: Not Found for url: https://api.crossref.org/works/10.22967/HCIS.2021.11.028